In [ ]:
"""
Created on Sun Nov 10 18:13:27 2024

@author: Jas
"""

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import Accuracy

In [3]:
# Step 1: Create a simple dataset of movie reviews
reviews = [
    "I love this movie", "This was a great film", "Fantastic story", "Amazing movie", "Really enjoyed it",
    "I hated this movie", "This movie was awful", "Worst film ever", "So boring", "Not good at all"
]
labels = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
print(reviews)

['I love this movie', 'This was a great film', 'Fantastic story', 'Amazing movie', 'Really enjoyed it', 'I hated this movie', 'This movie was awful', 'Worst film ever', 'So boring', 'Not good at all']


In [4]:
# Step 2: Tokenize the text data
tokenizer = Tokenizer(num_words=1000)
val=tokenizer.fit_on_texts(reviews)
X = tokenizer.texts_to_sequences(reviews)
print(X)
# Step 3: Pad the sequences to make them the same length
X = pad_sequences(X, padding='post', maxlen=10)
print(X)
# # Step 4: Create the Transformer model using Functional API
embed_dim = 64  # Embedding dimension
num_heads = 2   # Number of attention heads
num_layers = 2  # Number of encoder layers

[[3, 6, 1, 2], [1, 4, 7, 8, 5], [9, 10], [11, 2], [12, 13, 14], [3, 15, 1, 2], [1, 2, 4, 16], [17, 5, 18], [19, 20], [21, 22, 23, 24]]
[[ 3  6  1  2  0  0  0  0  0  0]
 [ 1  4  7  8  5  0  0  0  0  0]
 [ 9 10  0  0  0  0  0  0  0  0]
 [11  2  0  0  0  0  0  0  0  0]
 [12 13 14  0  0  0  0  0  0  0]
 [ 3 15  1  2  0  0  0  0  0  0]
 [ 1  2  4 16  0  0  0  0  0  0]
 [17  5 18  0  0  0  0  0  0  0]
 [19 20  0  0  0  0  0  0  0  0]
 [21 22 23 24  0  0  0  0  0  0]]


In [5]:
inputs = tf.keras.Input(shape=(10,))  # Input layer (sequence of integers)
embedding_layer = Embedding(input_dim=1000, output_dim=embed_dim, input_length=10)(inputs)

C:\Users\amang\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Positional Encoding: Adding simple positional encoding to the embeddings
position_embedding = tf.keras.layers.Embedding(input_dim=10, output_dim=embed_dim)(tf.range(10))
print(position_embedding)
embedding_with_positional_encoding = embedding_layer + position_embedding
print(embedding_with_positional_encoding)
# Encoder layers using MultiHeadAttention
x = embedding_with_positional_encoding  # Initial input to the Transformer encoder
for _ in range(num_layers):
      attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)  # Using self-attention
      attention_output = LayerNormalization()(attention_output + x)  # Residual connection
      attention_output = Dropout(0.1)(attention_output)

tf.Tensor(
[[-3.41584906e-02  4.70875539e-02  4.98977341e-02 -3.08175571e-02
   4.92170192e-02  4.63818796e-02 -3.05662286e-02 -7.57654756e-03
  -2.26571914e-02 -3.33640203e-02 -2.32733022e-02 -1.10966340e-02
   6.06539100e-03 -3.44159380e-02  3.67386267e-03 -1.57976747e-02
   1.00117214e-02  3.68866362e-02 -3.12434193e-02 -3.93682607e-02
  -4.85066064e-02 -3.58324125e-03 -1.52924657e-02  2.90969051e-02
   2.29766704e-02  1.58852600e-02  1.54667608e-02  4.54081930e-02
  -3.51678208e-03  4.16622795e-02  3.78173105e-02 -2.11986303e-02
   3.11143883e-02  1.11247227e-03  3.24110501e-02 -3.70546803e-02
  -1.55471936e-02 -4.52183969e-02 -2.91136391e-02 -2.55028605e-02
  -4.15469632e-02  3.88257764e-02 -2.70545613e-02 -7.06039742e-03
  -7.36974552e-03 -2.92605292e-02  3.23269404e-02  3.50591205e-02
  -2.92478204e-02 -1.45514831e-02 -6.85628504e-03  1.63729228e-02
   1.68222673e-02 -4.21718135e-02 -1.27538294e-03 -2.51648314e-02
   2.41911411e-03 -1.81071870e-02  2.03178637e-02 -1.46273598e-02

In [7]:
#      # Feed Forward Network
ff_output = Dense(64, activation='relu')(attention_output)
ff_output = Dense(embed_dim)(ff_output)
x = LayerNormalization()(ff_output + attention_output)  # Residual connection

In [8]:
# Pool the output (flatten and take the average over all tokens)
x = GlobalAveragePooling1D()(x)

In [9]:
# Final classification layer
output = Dense(1, activation='sigmoid')(x)

In [10]:
# Build the model
model = Model(inputs, output)

In [11]:
# # Step 5: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Accuracy()])

In [12]:
# Step 6: Train the model
history = model.fit(X, labels, epochs=10, verbose=1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.0000e+00 - loss: 0.7347
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.0000e+00 - loss: 0.6537
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.0000e+00 - loss: 0.6821
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.0000e+00 - loss: 0.6280
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.0000e+00 - loss: 0.5769
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.0000e+00 - loss: 0.5844
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.5822   
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step - accuracy: 0.0000e+00 - loss: 0.5761
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.0000e+00 - loss: 0.4980
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.0000e+00 - loss: 0.5042


In [13]:
# Step 7: Evaluate the model
train_loss, train_accuracy = model.evaluate(X, labels, verbose=0)
print(f'Training Accuracy: {train_accuracy}')
print(f'Training loss: {train_loss}')

Training Accuracy: 0.0
Training loss: 0.4830702841281891


In [14]:
# Step 8: Make a prediction with a new movie review
test_review = ["I really liked the movie"]
test_review_seq = tokenizer.texts_to_sequences(test_review)  # Tokenize the test review
test_review_pad = pad_sequences(test_review_seq, padding='post', maxlen=10)  # Pad the sequence

In [15]:
prediction = model.predict(test_review_pad)  # Get the prediction
print(f'Prediction for the new review: {"Positive" if prediction[0][0] > 0.5 else "Negative"}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
Prediction for the new review: Positive
